# Даны шесть объектов, каждый определяется четырьмя признаками. Разбить объекты на 3 кластера методом k-средних. Первоначально последние два объекта образуют начальные кластеры, метрика – квадрат евклидова расстояния:
X1(0; 1, 0, -1), Х2(-2; 1,-2, -2), Х3(1, 0; 1, 1), Х4 (-1, 1; 3, 0), Х5(-1, 0; 3, 1), Х6 (-2, 0;-2, -1).


In [ ]:
import random

In [ ]:
# Делаем массив points со всеми точками. Каждая точка - массив ее координат

x1 = [0, 1, 0, -1]
x2 = [-2, 1,-2, -2]
x3 = [1, 0, 1, 1]
x4 = [-1, 1, 3, 0]
x5 = [-1, 0, 3, 1]
x6 = [-2, 0,-2, -1]

points = [x1, x2, x3, x4, x5, x6]

In [ ]:
# Напишем функцию евклидова расстояния между двумя точками

def euclidean_distance(point_1, point_2):
	distance = 0.0
	for i in range(len(point_1)):
		distance += (point_1[i] - point_2[i])**2
	return (distance)**0.5

In [ ]:
# Функция которая на вход получает точки, и центры кластеров, и определяет к какому кластеру относится точка
def k_means_clustering_one_move(data, cluster_centers):

  clustering = []

  for i in range(len(cluster_centers)):
    clustering.append([])

  for _ in range(len(data)):
    equalizer = []
    for i in range(len(cluster_centers)):
      dist = euclidean_distance(data[_], cluster_centers[i])
      equalizer.append(dist)
    cluster_name = equalizer.index(min(equalizer))
    clustering[cluster_name].append(data[_])

  return clustering # массив, в котором n кластеров и каждая точка в своем кластере

In [ ]:
# расчет среднего внутрикластерного и межкластерного расстояний (по формулам из документа К.С. Зайцева  - https://education.yandex.ru/handbook/ml/article/klasterizaciya)

def inter_cluster_distance(clusters):
    total_distance = 0.0
    num_clusters = len(clusters)

    if num_clusters == 0:
        return 0.0

    for cluster in clusters:
        n = len(cluster)
        if n == 0:
            continue

        intra_distance = sum(euclidean_distance(point_1, point_2) for point_1 in cluster for point_2 in cluster)
        total_distance += intra_distance / (n * n)

    return total_distance / num_clusters


def intra_cluster_distance(centers):
    total_distance = 0
    num_clusters = len(centers)
    count = 0

    for i in range(num_clusters):
        for j in range(i + 1, num_clusters):
            total_distance += euclidean_distance(centers[i], centers[j])
            count += 1

    return total_distance / count if count > 0 else 0

In [ ]:
# В задаче странно заданы центры - 2 выбираются из объектов массива, 1 случайно, написали код в функции, который это и реализует
def k_means_clustering(data, n_clusters=3, eps = 0.0001):

  centers = []

  centers.append(data[-1])
  centers.append(data[-2]) # те самые 2 изначальных центра

  data_for_centers = data.copy()
  data_for_centers = data_for_centers[:-2] # уменьшаем выборку для определения 3го центра

  for _ in range(n_clusters - 2):
    new_center = random.choice(data_for_centers)# определили 3й центр
    centers.append(new_center) # добавили его

  counter = 0

  while True: # бесконечный цикл, в котором будет рассчитываться кластеризация до тех пор, пока разница между старым и новым положением координат центра не будет меньше эпсилон

    one_move_result = k_means_clustering_one_move(data, centers) # разделяем точки на класеры функцией выше

    centroids = [] # или новые предполагаемые центры

    for cluster in one_move_result: # рассчитываем центры - для каждого кластера прохожусь по всем его элементам и создаю самый центральный элемент (среднее арифм. координат) это и есть новый центр
        num_points = len(cluster)
        num_coords = len(cluster[0])

        centroid = []
        for i in range(num_coords):
            coord_sum = sum(point[i] for point in cluster)
            centroid.append(coord_sum / num_points)

        centroids.append(centroid)

    a = 0
    for i in range(n_clusters):
      a += euclidean_distance(centroids[i], centers[i])

    if a <= eps: # тут прекращение цикла
        break

    centers = centroids

    # еще метрики посчитаем, которые необходимо
    print(f"Мехкластерное расстояние: {intra_cluster_distance(centers)}, внутрикластерное расстояние: {inter_cluster_distance(one_move_result)}, итерация: {counter}")
    counter += 1

  return one_move_result, centers

In [ ]:
res, cen = k_means_clustering(points)

Мехкластерное расстояние: 3.662617552259634, внутрикластерное расстояние: 1.0485124701694568, итерация: 0
Мехкластерное расстояние: 3.668130973201471, внутрикластерное расстояние: 0.7842147305649726, итерация: 1
